<a href="https://colab.research.google.com/github/juliovicenzi/Benchmarks/blob/master/axur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Axur selection process


In [1]:
!pip install transformers=='4.49.0' datasets=='3.4.1' peft=='0.14.0' mlflow=='2.21.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## 1. Data collection

In [9]:
from datasets import load_dataset, DatasetDict

RANDOM_SEED=42

def load_alpaca_dataset(
        num_samples: int = 6000,
        test_size: float = 0.5,
        seed: int = RANDOM_SEED
    ) -> DatasetDict:
    """Reads the alpaca dataset from huggingface,
    and splits it into train and test.

    Args:
        num_samples: the number of rows to retrieve from the dataset
        test_size: the train/test split. Must be in [0,1]
        seed: random seed to use during shuffling
    Returns:
        the downloaded dataset split into train and test
    """
    alpaca_data = load_dataset(
        "tatsu-lab/alpaca",
        split=f"train[0:{num_samples}]"
    )

    shuffled_split_data = alpaca_data.train_test_split(
        test_size=0.5,
        shuffle=True,
        seed=RANDOM_SEED
    )
    return shuffled_split_data

data = load_alpaca_dataset()

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 3000
    })
})

In [16]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(RANDOM_SEED)
generator("What is the capital of Germany?")


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'What is the capital of Germany? What is Germany that is under German rule? What is the population, the size of Germany, that has been reduced to the lowest level, the number of people in Germany?\n\nLet us see. Today,'}]

In [7]:
def fine_tune_model(model, train_data, eval_data, epochs: int, learning_rate: float):
    pass

In [8]:
def evaluate_model(model, eval_data):
    pass

NameError: name 'Dataset' is not defined

# 3. How would you group questions from the train and test datasets that deal with similar subjects?

First, we can use sentence embedding to get vector representations of the questions.

From the embedding, we can perform similarity grouping either using *similarity search*, like [FASS](https://ai.meta.com/tools/faiss/).

A more traditional but apporach could make use K-means, or any other clustering algorithm to find similarities in the questions based on their sentence embeddings.